In [5]:
import numpy as np
import matplotlib.pyplot as plt
import emcee
import corner

In [6]:
def Likelihood(mu,n,b,s):
    
    l = mu*s + b
    
    L = np.exp( -l ) * l**n / np.math.factorial(int(n))
    
    return L

In [7]:
def JointLogLikelihood(mu,data):
    
    N = data.shape[0]
    
    n = data[:,0]
    b = data[:,1]
    s = data[:,2]
    
    JointL = 0.
    
    for c in range(N):
        JointL += np.log(Likelihood(mu,n[c],b[c],s[c]))
        
    return JointL

In [8]:
n = np.array([1,0,0])
b = np.array([0,0,0])
s = np.array([1,2,1])

N = n.shape[0]

data = np.zeros((N,3))

data[:,0] = n
data[:,1] = b
data[:,2] = s

data

array([[1., 0., 1.],
       [0., 0., 2.],
       [0., 0., 1.]])

In [9]:
mu = np.linspace(0.001,2.,1000)
Posterior = np.zeros_like(mu)